In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\International Food\\Desktop\\New folder (2)\\kidney deasies prediction\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path 
@dataclass
class Trainingconfig:
    root_dir: Path
    trained_model_path:Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch: int
    params_is_augmentation: bool
    params_image_size: list

In [5]:
from src.ccnClassifier.constants import *
from src.ccnClassifier.utils.common import read_yaml,create_directories

In [6]:
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(self,
    config_filepath= CONFIG_FILE_PATH,
    params_filepath =PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
        
    def get_training_config(self)->Trainingconfig:
        training=self.config.training
        training_data=self.config.data_ingestion
        params=self.params
        prepare_base_model_path=self.config.prepare_base_model
        create_directories(
            [Path(training.root_dir)]
        )
        
        training_config=Trainingconfig(
            root_dir = Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path= Path(prepare_base_model_path.updated_base_model_path),
            training_data=Path(os.path.join(training_data.unzip_dir,"kidney-ct-scan-image")),
            params_epochs=params.EPOCHES,
            params_batch=params.BATCH_SIZE,
            params_is_augmentation=params.AUGNENTATION,
            params_image_size=params.IMAGE_SIZE
            
            )
        
        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:

class Training:
    def __init__(self,config: Trainingconfig):
        self.config=config
        
    def get_base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model_path
            )
        
        
    def train_valid_generator(self):
        
        datagenerator_kwargs=dict(rescale=1./255,
                                validation_split=0.20)
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            interpolation="bilinear"
        )
            
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=40,
            horizontal_flip=True,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            **datagenerator_kwargs
            )
            
            
        else:
            train_datagenerator= valid_datagenerator
            
        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset=None,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(path: Path,model: tf.keras.models):
        model.save(path)
        
        
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
                        validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        
         
        

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

{2024-03-20 14:26:48,721:INFO:common:yaml_file:config\config.yaml loaded sucessfully}
{2024-03-20 14:26:48,994:INFO:common:yaml_file:params.yaml loaded sucessfully}
{2024-03-20 14:26:49,184:INFO:common:created directory at:artifacts}
{2024-03-20 14:26:49,186:INFO:common:created directory at:artifacts\training}
Found 93 images belonging to 2 classes.
Found 465 images belonging to 2 classes.
14/14 [==============================] - 6265s 467s/step - loss: 13.2317 - accuracy: 0.5173 - val_loss: 3.5103 - val_accuracy: 0.7500
